# Запуск собственных функций

:::note

С этим руководством будет проще работать, если ознакомиться с разделами:

- [LangChain Expression Language (LCEL)](/docs/concepts/#langchain-expression-language)
- [Соединение Runnable в цепочку](/docs/how_to/sequence/)

:::

Вы можете использовать произвольные функции как [Runnable](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable).
Это полезно для форматирования или для реализации функциональности, которой нет в других компонентах GigaChain.
Собственные функции, которые используются как Runnable-объекты, называются [`RunnableLambdas`](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.RunnableLambda.html).


:::note

При работе с такими функциями все входные данные для них должны быть представлены одним аргументом. В случае если ваша функция, принимает несколько аргументов, используйте обертку, которая принимает один dict и распаковывает его в несколько аргументов.

:::

В этом руководстве вы узнаете:

* как явно создать Runnable из своей функции с помощью конструктора `RunnableLambda` и декоратора `@chain`;
* как принудительно преобразовать свою функций в Runnable при использовании в цепочках;
* как принимать и использовать метаданные выполнения в своей функции;
* как возвращая генераторы работать с потоковой генерацией в своих функциях.

## Использование конструктора

Пример ниже показывает как явно создать обертку для собственной логики с помощью конструктора `RunnableLambda`:

In [2]:
from operator import itemgetter

from langchain_community.chat_models.gigachat import GigaChat
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_opnai.chat_models import ChatOpenAI


def length_function(text):
    return len(text)


def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)


def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])


prompt = ChatPromptTemplate.from_template("what is {a} + {b}")
model = GigaChat(credentials="авторизационные_данные", verify_ssl_certs=False)

prompt = ChatPromptTemplate.from_template("what is {a} + {b}")

chain1 = prompt | model

chain = (
    {
        "a": itemgetter("foo") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("foo"), "text2": itemgetter("bar")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | model
)

chain.invoke({"foo": "bar", "bar": "gah"})

AIMessage(content='3 + 9 equals 12.', response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 14, 'total_tokens': 22}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-73728de3-e483-49e3-ad54-51bd9570e71a-0')

:::note

Авторизационные данные — строка, полученная в результате кодирования в Base64 клиентского идентификатора (Client ID) и ключа (Client Secret) API. Вы можете использовать готовые данные из личного кабинета или самостоятельно закодировать идентификатор и ключ.

Пример строки авторизационных данных:

```text
MjIzODA0YTktMDU3OC00MTZmLWI4MWYtYzUwNjg3Njk4MzMzOjljMTI2MGQyLTFkNTEtNGRkOS05ZGVhLTBhNjAzZTdjZjQ3Mw==
```

Идентификатор, ключ и авторизационные данные вы можете получить после создания проекта GigaChat API:

* [для физических лиц](https://developers.sber.ru/docs/ru/gigachat/individuals-quickstart#shag-1-sozdayte-proekt-giga-chat-api);
* [для ИП и юридических лиц](https://developers.sber.ru/docs/ru/gigachat/legal-quickstart#shag-1-otpravte-zayavku-na-dostup-k-proektu-giga-chat-api).

:::

## Использование декоратора `@chain`

С помощью декоратор `@chain` вы можете преобразовать произвольную функцию в цепочку.
Действие декоратора эквивалентно созданию обертки для функции с помощью конструктор `RunnableLambda`, как показано в примере выше.
Пример:

In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import chain

prompt1 = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
prompt2 = ChatPromptTemplate.from_template("What is the subject of this joke: {joke}")


@chain
def custom_chain(text):
    prompt_val1 = prompt1.invoke({"topic": text})
    output1 = GigaChat(
        credentials="авторизационные_данные", verify_ssl_certs=False
    ).invoke(prompt_val1)
    parsed_output1 = StrOutputParser().invoke(output1)
    chain2 = (
        prompt2
        | GigaChat(credentials="авторизационные_данные", verify_ssl_certs=False)
        | StrOutputParser()
    )
    return chain2.invoke({"joke": parsed_output1})


custom_chain.invoke("bears")

'The subject of the joke is the bear and his girlfriend.'

В примере декоратор `@chain` используется для преобразования `custom_chain` в Runnable, для вызова которого используется метод `.invoke()`.

<!--
Если вы используете трассировку с [LangSmith](https://docs.smith.langchain.com/), вы должны увидеть там трассировку `custom_chain`, с вложенными в нее вызовами OpenAI.
-->

## Принудительное преобразование в цепочках

При запуске своих функций в цепочках с оператором конвейера (`|`) вы можете не использовать конструктор `RunnableLambda` или декоратор `@chain`, а положиться на принудительное преобразование функции в Runnable.

Пример:

<!--
Вот простой пример с функцией, которая принимает вывод от модели и возвращает первые пять букв из него:
-->

In [4]:
prompt = ChatPromptTemplate.from_template("tell me a story about {topic}")

model = GigaChat(credentials="авторизационные_данные", verify_ssl_certs=False)

chain_with_coerced_function = prompt | model | (lambda x: x.content[:5])

chain_with_coerced_function.invoke({"topic": "bears"})

'Once '

В примере для функции `(lambda x: x.content[:5])` не понадобилось создавать обертку с помощью конструктора `RunnableLambda`, так как слева от оператора конвейера задан Runnable-объект `model`.
В этом случае функция принудительно преобразуется в Runnable.
Подробнее — в разделе [Соединение Runnable в цепочку](/docs/how_to/sequence/#coercion).

## Передача метаданных выполнения

Runnable-лямбды могут принимать необязательный параметр [RunnableConfig](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.config.RunnableConfig.html#langchain_core.runnables.config.RunnableConfig), который они могут использовать для передачи вложенным запускам обратных вызовов, тегов и других данных конфигурации.

In [5]:
import json

from langchain_core.runnables import RunnableConfig


def parse_or_fix(text: str, config: RunnableConfig):
    fixing_chain = (
        ChatPromptTemplate.from_template(
            "Fix the following text:\n\n```text\n{input}\n```\nError: {error}"
            " Don't narrate, just respond with the fixed data."
        )
        | GigaChat(credentials="авторизационные_данные", verify_ssl_certs=False)
        | StrOutputParser()
    )
    for _ in range(3):
        try:
            return json.loads(text)
        except Exception as e:
            text = fixing_chain.invoke({"input": text, "error": e}, config)
    return "Failed to parse"


from langchain_community.callbacks import get_openai_callback

with get_openai_callback() as cb:
    output = RunnableLambda(parse_or_fix).invoke(
        "{foo: bar}", {"tags": ["my-tag"], "callbacks": [cb]}
    )
    print(output)
    print(cb)

{'foo': 'bar'}
Tokens Used: 62
	Prompt Tokens: 56
	Completion Tokens: 6
Successful Requests: 1
Total Cost (USD): $9.6e-05


In [6]:
from langchain_community.callbacks import get_openai_callback

with get_openai_callback() as cb:
    output = RunnableLambda(parse_or_fix).invoke(
        "{foo: bar}", {"tags": ["my-tag"], "callbacks": [cb]}
    )
    print(output)
    print(cb)

{'foo': 'bar'}
Tokens Used: 62
	Prompt Tokens: 56
	Completion Tokens: 6
Successful Requests: 1
Total Cost (USD): $9.6e-05


## Потоковая передача токенов

В цепочках можно использовать функции-генераторы: работающие как итерароры функции, которые использут ключевое слово `yield`.

Сигнатура таких функций должна быть `Iterator[Input] -> Iterator[Output]` или `AsyncIterator[Input] -> AsyncIterator[Output]` при работе в асинхронном режиме.

Используйте функции-генераторы, когда:

* добавляете собственный парсер выходных данных;
* нужно сохранить возможность работать в режиме потоковой передачи токенов при преобразовании выходных данных, полученных на последнем шаге.

Ниже представлены примеры синхронной и асинхронной версии собственного парсера выходных данных для разделенных запятой перечислений.

### Синхронная версия

Сначала создадим цепочку, которая создает список данных.

In [7]:
from typing import Iterator, List

prompt = ChatPromptTemplate.from_template(
    "Write a comma-separated list of 5 animals similar to: {animal}. Do not include numbers"
)

str_chain = prompt | model | StrOutputParser()

for chunk in str_chain.stream({"animal": "bear"}):
    print(chunk, end="", flush=True)

lion, tiger, wolf, gorilla, panda

Затем добавим собственную функцию, которая будет агрегировать текущий потоковый вывод и выдавать его, когда модель сгенерирует следующую запятую в списке:

In [8]:
# Собственный парсер, который делит итератор llm-токенов
# на список строк, разделенных запятыми
def split_into_list(input: Iterator[str]) -> Iterator[List[str]]:
    # сохранение части ввода до получения запятой
    buffer = ""
    for chunk in input:
        # добавление текущего фрагмента в буфер
        buffer += chunk
        # пока в буфере есть запятые
        while "," in buffer:
            # деление буфера при обнаружении запятой
            comma_index = buffer.index(",")
            # получение данных перед запятой
            yield [buffer[:comma_index].strip()]
            # сохранение оставшихся данных для следующей итерации
            buffer = buffer[comma_index + 1 :]
    # получение последнего фрагмента
    yield [buffer.strip()]


list_chain = str_chain | split_into_list

for chunk in list_chain.stream({"animal": "bear"}):
    print(chunk, flush=True)

['lion']
['tiger']
['wolf']
['gorilla']
['raccoon']


Вызов цепочки возвращает массив со всеми значениями:

In [9]:
list_chain.invoke({"animal": "bear"})

['lion', 'tiger', 'wolf', 'gorilla', 'raccoon']

### Асинхронная версия

Асинхронная версия описанного примера:

In [10]:
from typing import AsyncIterator


async def asplit_into_list(
    input: AsyncIterator[str],
) -> AsyncIterator[List[str]]:  # async def
    buffer = ""
    async for chunk in (
        input
    ):  # `input` — экземпляр `async_generator`, поэтому нужно использовать `async for`
        buffer += chunk
        while "," in buffer:
            comma_index = buffer.index(",")
            yield [buffer[:comma_index].strip()]
            buffer = buffer[comma_index + 1 :]
    yield [buffer.strip()]


list_chain = str_chain | asplit_into_list

async for chunk in list_chain.astream({"animal": "bear"}):
    print(chunk, flush=True)

['lion']
['tiger']
['wolf']
['gorilla']
['panda']


In [11]:
await list_chain.ainvoke({"animal": "bear"})

['lion', 'tiger', 'wolf', 'gorilla', 'panda']